





## This snnipet code show the implementation of Logistic Regression using Tensorflow.
### Here we produce imaginary data using data class 

In [5]:
import tensorflow as tf
from util.data import *
import numpy as np


### Data  and Parameters -----------------------------------------------------------


In [6]:
learning_rate=0.01
epoch_number=200
Sample_Number=1000
Feature_Number=1024
batch_size=256
Diplay_step=100

#### Instead of using the most popular data set for deep learning (mnist), I decided to create a toy data set. Each sample in this new  data set is the binary from of numbers in range(0,Sample_Number) in shuffeled form. Also the labels are integers in range (0,9)  which equal to (number of ones in sample) mode 10. In addition all labels are in one-hot format
#### After creating a full data set, it's devided into Test and Train samples with their respective labels according to the Test_percentage.


In [11]:
DataObj=Data(data_len=Feature_Number,Sample_Number=Sample_Number,Test_percentage=40)
Train_x,Train_y,Test_x,Test_y=DataObj.CNN_data()
Number_of_batches=int(len(Train_x)/batch_size)

### Structure Definition---------------------------------------------------


In [17]:
with tf.name_scope("Inputs"):
       
    X=tf.placeholder(dtype=tf.float32,shape=[None,Feature_Number],name='x_inputs')
    Y=tf.placeholder(dtype=tf.float32,shape=[None,10],name='y') # Number of classes is 10
with tf.name_scope('Weights'):
    
    W=tf.Variable(tf.random_normal([Feature_Number,10],stddev=0.01),name='W')
with tf.name_scope('biases'):
    b=tf.Variable(tf.zeros([10]),name='b')

def model(X,W,b):
    with tf.name_scope('Model'):
        result=tf.add(tf.matmul(X,W,name='matmul'),b,name='bias_addition') # Matrix multiplication
        return tf.nn.softmax(result,name='softmax')
# Prediction is of the vector of probabilities form shows the confidence of each classs
Prediction=model(X,W,b)

# Minimize error using cross entropy with logits
with tf.name_scope('cost'):
    cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Prediction,labels=Y),name='reduce_mean')
    
# Minimize error using cross entropy
#cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(Prediction), reduction_indices=1))

with tf.name_scope('train'):
    train_step=tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

predicted_label=tf.equal(tf.argmax(Prediction, 1), tf.argmax(Y, 1)) 
with tf.name_scope('Accuracy'):
    Accuracy=tf.reduce_mean(tf.cast(predicted_label, "float"),name='Accuracy')



### Running Graph-----------------------------------------------------------------------

In [18]:
with tf.Session() as sess:
    writer=tf.summary.FileWriter('log/',sess.graph)
    sess.run(tf.global_variables_initializer())
    for i in range(epoch_number):
        avg_cost=0
        for j in range(Number_of_batches):            
            Trx,Try=next_batch(batch_size, Train_x,Train_y)
            sess.run(train_step,feed_dict={X:Trx,Y:Try})
            avg_cost+=sess.run(cost,feed_dict={X:Trx,Y:Try})/Number_of_batches
        
        # Shows Cost and accuracy on Train and Test data 
        if i%Diplay_step==0:
            Train_Accuracy=sess.run(Accuracy,feed_dict={X:Trx,Y:Try})
            Test_Accuracy=sess.run(Accuracy,feed_dict={X:Test_x,Y:Test_y})
        
            print ("Epoch: %03d/%03d cost: %.9f train_acc: %.3f test_acc: %.3f" 
                   % (i, epoch_number, avg_cost, Train_Accuracy, Test_Accuracy))
        

Epoch: 000/200 cost: 2.302622795 train_acc: 0.082 test_acc: 0.095
Epoch: 100/200 cost: 2.300885081 train_acc: 0.238 test_acc: 0.203
